In [13]:
import sys
sys.path.append(r'D:\Pixta\ultralytics_custom5')
import pandas as pd
import json
import os
import cv2
import torch
from ultralytics import YOLO
from matplotlib import pyplot as plt
import time
import numpy as np

In [14]:
info_label = {
    'age': ['20-30s', '40-50s', 'Baby', 'Kid', 'Senior', 'Teenager'],
    'race': ['Caucasian', 'Mongoloid', 'Negroid'],
    'masked': ['masked', 'unmasked'],
    'skintone': ['dark', 'light', 'mid-dark', 'mid-light'],
    'emotion': ['Anger', 'Disgust', 'Fear', 'Happiness', 'Neutral', 'Sadness', 'Surprise'],
    'gender': ['Female', 'Male']
}

In [15]:
def imshow_np_image(img):
    plt.imshow(img[:,:,::-1])    
    plt.show()

def tensor_to_float(lis):
    # lis = [int(np.round(i.cpu().detach().numpy()) for i in lis)]
    lis = [float(i.cpu().detach().numpy()) for i in lis]
    return lis
    
def process_out_detect(img, results):
    bbox_lis = []
    for r in results:
        boxes = r.boxes
        for i, box in enumerate(boxes):
            x, y, w, h = box.xywh[0]

            bbox_lis.append(tensor_to_float([x-w/2, y-h/2, w, h]))

            cv2.rectangle(img, (int(x-w/2),int(y-h/2)), (int(x+w/2),int(y+h/2)), (0,255,0), 3)
            # cv2.putText(img, str(i+1) ,(x1,y1+20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 1)
    return bbox_lis

def sample_values(lis):
    print(lis)
    lis = list(lis)
    while len(lis) < 30:
        lis.append(-1)
    return lis

def convert_to_text(key, index):
    return info_label[key][index]

def procees_out_cls(results, index_box):
    preds = {}
    num_cate = iter([6,3,2,4,7,2])
    for key in results.keys():
        cur_cate = next(num_cate)
        pred = results[key]
        pred = pred[:, (index_box * cur_cate):((index_box + 1) * cur_cate)]
        pred = torch.softmax(pred, dim= -1)
        pred = torch.argmax(pred, dim= -1)
        index = int(pred.cpu().detach().numpy())
        preds[key] = convert_to_text(key, index)
    return preds

In [16]:
image_id = json.load(open('file_name_to_image_id_preal.json'))

In [17]:
yolo = YOLO('../pbest.pt')
# yolo = YOLO('pretrained\yolov8s\\pbest.pt')
img = cv2.imread('../public_test/100147591.jpg')
a = yolo('../public_test/100147591.jpg')


image 1/1 d:\Pixta\ultralytics_custom5\submit-related\..\public_test\100147591.jpg: 224x320 1 face, 7.0ms
Speed: 1.0ms preprocess, 7.0ms inference, 1.0ms postprocess per image at shape (1, 3, 224, 320)


In [18]:
path = '../../private_test_data/private_test_data/'
all_preds = []
times = []
names = os.listdir(path)
for index, name in enumerate(image_id):
    print(index)
    name_id = image_id[name]
    results = yolo(path + name)

    bbox_lis = []
    for r in results[0]:
        boxes = r.boxes
        for i, box in enumerate(boxes):
            x, y, w, h = box.xywh[0]
            bbox_lis.append(tensor_to_float([x-w/2, y-h/2, w, h]))

    preds = {}
    for i in range(len(bbox_lis)):
        num_cate = iter([6,3,2,4,7,2])
        for key in results[1].keys():
            cur_cate = next(num_cate)
            pred = results[1][key]
            pred = pred[:, (i * cur_cate):((i + 1) * cur_cate)]
            pred = torch.softmax(pred, dim= -1)
            pred = torch.argmax(pred, dim= -1)
            index = int(pred.cpu().detach().numpy())
            preds[key] = convert_to_text(key, index)
        all_preds.append([name, bbox_lis[i], name_id, preds['race'], preds['age'], preds['emotion'], preds['gender'], preds['skintone'], preds['masked']])

    

0



image 1/1 d:\Pixta\ultralytics_custom5\submit-related\..\..\private_test_data\private_test_data\private_test_img_0.jpg: 320x320 1 face, 9.0ms
Speed: 1.0ms preprocess, 9.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 320)
1

image 1/1 d:\Pixta\ultralytics_custom5\submit-related\..\..\private_test_data\private_test_data\private_test_img_1.jpg: 320x320 1 face, 9.0ms
Speed: 1.0ms preprocess, 9.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 320)
2

image 1/1 d:\Pixta\ultralytics_custom5\submit-related\..\..\private_test_data\private_test_data\private_test_img_10.jpg: 320x320 2 faces, 10.0ms
Speed: 1.0ms preprocess, 10.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 320)
3

image 1/1 d:\Pixta\ultralytics_custom5\submit-related\..\..\private_test_data\private_test_data\private_test_img_100.jpg: 320x320 1 face, 7.0ms
Speed: 1.0ms preprocess, 7.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 320)
4

image 1/1 d:\Pixta\ultralytics

In [19]:
all_preds[0]

['private_test_img_0.jpg',
 [247.7486572265625, 176.21273803710938, 577.501953125, 811.5734252929688],
 1,
 'Caucasian',
 '20-30s',
 'Happiness',
 'Female',
 'light',
 'unmasked']

In [20]:
# file_name,bbox,image_id,race,age,emotion,gender,skintone,masked
final = pd.DataFrame(columns= ['file_name', 'bbox', 'image_id', 'race', 'age', 'emotion', 'gender', 'skintone', 'masked'], data= all_preds)

In [24]:
final

,file_name,bbox,image_id,race,age,emotion,gender,skintone,masked
0,private_test_img_0.jpg,"[247.7486572265625, 176.21273803710938, 577.50...",1,Caucasian,20-30s,Happiness,Female,light,unmasked
1,private_test_img_1.jpg,"[246.85018920898438, 188.20379638671875, 597.7...",2,Caucasian,40-50s,Happiness,Female,light,unmasked
2,private_test_img_10.jpg,"[214.75299072265625, 172.22708129882812, 583.6...",3,Mongoloid,20-30s,Happiness,Female,mid-light,unmasked
3,private_test_img_10.jpg,"[454.61016845703125, 160.17266845703125, 353.4...",3,Caucasian,20-30s,Happiness,Male,light,unmasked
4,private_test_img_100.jpg,"[241.22152709960938, 144.54302978515625, 584.5...",4,Caucasian,20-30s,Happiness,Female,light,unmasked
...,...,...,...,...,...,...,...,...,...
6484,private_test_img_996.jpg,"[215.125732421875, 229.02227783203125, 612.094...",5288,Caucasian,Kid,Happiness,Female,light,unmasked
6485,private_test_img_997.jpg,"[241.40240478515625, 158.73553466796875, 567.0...",5289,Caucasian,Kid,Neutral,Female,light,unmasked
6486,private_test_img_998.jpg,"[692.5927734375, 830.2623291015625, 119.363586...",5290,Caucasian,20-30s,Happiness,Female,mid-light,unmasked
6487,private_test_img_998.jpg,"[285.4703369140625, 999.61572265625, 73.110504...",5290,Caucasian,20-30s,Neutral,Male,light,unmasked


In [25]:
final['gender'].unique()

array(['Female', 'Male'], dtype=object)

In [23]:
final.to_csv('answer.csv', index= False)